# Giriş İşlemleri

In [ ]:
%%capture
!pip install kagglehub pandas datasets scikit-learn torch transformers[torch]

In [ ]:
import kagglehub
import pandas as pd
import os
from datasets import Dataset
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from transformers import AutoModelForSequenceClassification, AutoTokenizer
from transformers import Trainer, TrainingArguments
import numpy as np
import torch

In [ ]:
def is_colab():
    try:
        import google.colab
        return True
    except ImportError:
        return False

In [ ]:
# Kök dizin belirleme
if is_colab():
    from google.colab import drive
    drive.mount('/content/drive')
    kok_dizin = "/content/drive/MyDrive/spam_message_classifier_tr"
else:
    kok_dizin = os.getcwd()
print(f"Kök dizin: {kok_dizin}\n Not: eğer colab kullanıyorsanız, dizini değiştirmeniz gerekir.")

In [ ]:
model_save_path = os.path.join(kok_dizin, "egitilen_model")
model_cikti_dizini = os.path.join(kok_dizin, "kilometre_tasi")
model_log_dizini = os.path.join(kok_dizin, "log")
eval_sonuc_json_path = os.path.join(model_log_dizini, "eval_sonuc.json")
train_sonuc_json_path = os.path.join(model_log_dizini, "train_sonuc.json")
if not os.path.exists(model_cikti_dizini):
    os.makedirs(model_cikti_dizini)
if not os.path.exists(model_save_path):
    os.makedirs(model_save_path)

In [ ]:
cihaz = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Cihaz: {cihaz}")

# Veri Kümesi Okuma

## Veri kümesi okuma fonksiyonu

In [ ]:
def veri_kumesi_oku(veri_kumesi_adi: str) -> pd.DataFrame:
    """
    Kaggle veri kümesini indirir ve döndürür.

    Args:
        veri_kumesi_adi (str): Kaggle veri kümesinin yolu.
        gecici_yol (str): Geçici dosya yolu.
    Returns:
        pd.DataFrame: İndirilen veri kümesi.
    """
    # Kaggle veri kümesini indir
    gecici_yol = os.path.join(kagglehub.dataset_download(veri_kumesi_adi), "TurkishSMSCollection.csv")
    return pd.read_csv(gecici_yol, sep=";", encoding="utf-8")

## Veri kümesini oku

In [ ]:
# Veri kümesi bilgileri
dataset_name = "onurkarasoy/turkish-sms-collection"

veri_kumesi = veri_kumesi_oku(dataset_name)
veri_kumesi.head(10)  # İlk 10 satırı göster

## Veri kümesinin gereksiz kolonlarını silme

In [ ]:
tohum_degeri = 571
# gereksiz sütunları kaldır
veri_kumesi = veri_kumesi[['Message', 'Group']]
# veri kümesini karıştır
veri_kumesi.head(10)  # İlk 10 satırı göster

## Veri kümesini karıştırma

In [ ]:
karismis_veri_kumesi = veri_kumesi.sample(frac=1, random_state=tohum_degeri).reset_index(drop=True)
karismis_veri_kumesi.head(10)  # İlk 10 satırı göster

## Veri kümesini eğitim ve sınama olarak parçalama

In [ ]:
def veri_kumesini_oranli_bol(veri_kumesi, test_orani=0.1, dogrulama_orani=0.1, tohum_degeri=42):
    """
    Veri kümesini, Group kolonundaki değerlerin oranlarını koruyarak üçe böler.

    Args:
        veri_kumesi (pd.DataFrame): Bölünecek veri kümesi
        test_orani (float): Test veri kümesinin oranı (varsayılan: 0.1)
        dogrulama_orani (float): Doğrulama veri kümesinin oranı (varsayılan: 0.1)
        tohum_degeri (int): Random state değeri (tekrarlanabilirlik için)

    Returns:
        tuple: (egitim_veri_kumesi, dogrulama_veri_kumesi, test_veri_kumesi) - Oransal olarak bölünmüş veri kümeleri
    """
    # Benzersiz grup değerlerini bul
    gruplar = veri_kumesi['Group'].unique()

    egitim_liste = []
    dogrulama_liste = []
    test_liste = []

    # Her grup için ayrı ayrı bölme işlemi yap
    for grup in gruplar:
        # Sadece bu gruba ait verileri seç
        grup_verisi = veri_kumesi[veri_kumesi['Group'] == grup]

        # Bu grubu karıştır
        karisik_grup = grup_verisi.sample(frac=1, random_state=tohum_degeri)

        # Bölme noktalarını hesapla
        test_bolme_noktasi = int(len(karisik_grup) * test_orani)
        dogrulama_bolme_noktasi = int(len(karisik_grup) * dogrulama_orani)

        # Bu gruptan test, doğrulama ve eğitim verilerini ayır
        test_grubu = karisik_grup[:test_bolme_noktasi]
        dogrulama_grubu = karisik_grup[test_bolme_noktasi:test_bolme_noktasi+dogrulama_bolme_noktasi]
        egitim_grubu = karisik_grup[test_bolme_noktasi+dogrulama_bolme_noktasi:]

        # Listelere ekle
        test_liste.append(test_grubu)
        dogrulama_liste.append(dogrulama_grubu)
        egitim_liste.append(egitim_grubu)

    # Grupları birleştir
    sinama_veri_kumesi = pd.concat(test_liste).reset_index(drop=True)
    dogrulama_veri_kumesi = pd.concat(dogrulama_liste).reset_index(drop=True)
    egitim_veri_kumesi = pd.concat(egitim_liste).reset_index(drop=True)

    # Son bir karıştırma işlemi
    sinama_veri_kumesi = sinama_veri_kumesi.sample(frac=1, random_state=tohum_degeri).reset_index(drop=True)
    dogrulama_veri_kumesi = dogrulama_veri_kumesi.sample(frac=1, random_state=tohum_degeri).reset_index(drop=True)
    egitim_veri_kumesi = egitim_veri_kumesi.sample(frac=1, random_state=tohum_degeri).reset_index(drop=True)

    return egitim_veri_kumesi, dogrulama_veri_kumesi, sinama_veri_kumesi

In [ ]:
egitim_veri, dogrulama_veri, sinama_veri = veri_kumesini_oranli_bol(karismis_veri_kumesi, tohum_degeri=tohum_degeri)
egitim_veri.head(5)  # İlk 5 satırı göster
dogrulama_veri.head(5)
sinama_veri.head(5)  # İlk 5 satırı göster

## Veri kümelerini eğitime hazır hale getirme fonksiyonu

In [ ]:
# Veri kümesini hazırla
def verileri_hazirla(df, tokenizer):
    # Group değerlerini 0 ve 1'e dönüştür (Group=1 -> 0, Group=2 -> 1)
    # BERT sınıflandırma için 0-based indeksler kullanır
    labels = df["Group"].apply(lambda x: 0 if x == 1 else 1).tolist()

    # Dataset'e dönüştür
    dataset = Dataset.from_dict({
        "text": df["Message"].tolist(),
        "label": labels
    })

    # Tokenize işlemi
    def tokenize_function(examples):
        return tokenizer(
            examples["text"],
            padding="max_length",
            truncation=True,
            max_length=512,  # BERT için genellikle 512 kullanılır
        )

    tokenized_dataset = dataset.map(tokenize_function, batched=True)
    return tokenized_dataset

## Sınama sonucu hesaplama fonksiyonu

In [ ]:
# ölçüt hesaplama fonksiyonu
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, predictions, average='binary')
    acc = accuracy_score(labels, predictions)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

# Model Eğitim işlemleri

## Model yükleme

In [ ]:
model_adi = "dbmdz/bert-base-turkish-128k-cased"
model = AutoModelForSequenceClassification.from_pretrained(model_adi)
tokenizer = AutoTokenizer.from_pretrained(model_adi)
model.to(cihaz)

In [ ]:
# Eğitim ve test veri kümelerini hazırla
egitim_veri_kumesi = verileri_hazirla(egitim_veri, tokenizer)
dogrulama_veri_kumesi = verileri_hazirla(dogrulama_veri, tokenizer)
sinama_veri_kumesi = verileri_hazirla(sinama_veri, tokenizer)

## Eğitim argümanlarını ve eğiticiyi ayarla

In [ ]:
# Eğitim argümanlarını tanımla
training_args = TrainingArguments(
    output_dir=model_cikti_dizini,
    num_train_epochs=1,  # 1 epoch
    report_to=["tensorboard"],
    per_device_train_batch_size=8,
    per_device_eval_batch_size=1,
    gradient_accumulation_steps=16,
    warmup_steps=0,
    weight_decay=0.01,
    logging_dir=model_log_dizini,
    logging_steps=5,
    eval_strategy="steps",
    eval_steps=5,          # Her 5 adımda bir değerlendirme yapılacak
    save_strategy="steps",
    save_steps=5,          # Her 5 adımda bir model kaydedilecek
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    # Konsola çıktı için ek parametreler:
    logging_strategy="steps",
    logging_first_step=True,  # İlk adımın da loglanması için
)

# Eğiticiyi oluştur
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=egitim_veri_kumesi,
    eval_dataset=dogrulama_veri_kumesi,
    compute_metrics=compute_metrics,
)

## Eğitim ve sınama işlemleri

In [ ]:
# Modeli eğit
trainer.train()

In [ ]:
import json
train_results = trainer.state.log_history
with open(train_sonuc_json_path, "w") as f:
    json.dump(train_results, f, indent=4)

In [ ]:
# Değerlendirme
eval_results = trainer.evaluate(sinama_veri_kumesi)
print(f"Değerlendirme sonuçları: {eval_results}")

with open(eval_sonuc_json_path, "w") as f:
    json.dump(eval_results, f, indent=4)
print(f"Değerlendirme sonuçları {eval_sonuc_json_path} dosyasına kaydedildi.")

## Eğitilen modeli kaydetme

In [ ]:
# Eğitilen modeli kaydet
model.save_pretrained(model_save_path)
tokenizer.save_pretrained(model_save_path)